In [24]:
!pip install azureml-sdk

In [75]:
import os
import json
import requests

from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

In [46]:
# laoding the configuration file - standard way - use .env file and load_dotenv from python-dotenv module
config_file_path = "/home/azureuser/cloudfiles/code/Users/pemphokatsala/hackathon ai/config.json"

# Read JSON data into a dictionary
with open(config_file_path, 'r') as file:
    data = json.load(file)

subscription_id = data["subscription_id"]
resource_group = data["resource_group"]  
workspace_name = data["workspace_name"]
region = data["region"]

In [74]:
print(resource_group)
print(workspace_name)
print(region)

twonky
hackathon
eastus


In [73]:
# Load the existing workspace
ws = Workspace.get(name=workspace_name,
                   subscription_id=subscription_id,
                   resource_group=resource_group)

print(f'Workspace {workspace_name} loaded')

Workspace hackathon loaded


In [72]:
# Specify the path to your  model file
model_path = '/home/azureuser/cloudfiles/code/LR_model.pkl'

In [71]:
model_name='LR_model'

In [50]:
# Register the model in Azure Machine Learning
registered_model = Model.register(model_path=model_path, model_name=model_name, workspace=ws)

Registering model LR_model


In [81]:
# Create a Conda environment for your scikit-learn model
#conda_env = Environment('my-conda-env')
#conda_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])
# Create environment from yaml file
env = Environment.from_conda_specification(name='model-env', file_path='environment.yml')
print(env.python.conda_dependencies.serialize_to_string())

name: model-env
channels:
- conda-forge
- defaults
dependencies:
- python=3.8
- pip=20.0
- scikit-learn
- numpy
- pillow    # Add pillow as a conda dependency
- pip:
  - azureml-defaults
  - azureml-core
  - joblib
  - azure-storage-blob
  - inference-schema



In [83]:
# Create an InferenceConfig
inference_config = InferenceConfig(entry_script='/home/azureuser/cloudfiles/code/Users/pemphokatsala/hackathon ai/score.py', environment=conda_env)

In [70]:
# Specify deployment configuration for ACI
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

In [84]:
service = Model.deploy(workspace=ws,
                       name='lr-prediction-service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config)
service.wait_for_deployment(show_output=True)

/tmp/ipykernel_3180/3846008021.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-04-02 06:26:31+00:00 Registering the environment.
2025-04-02 06:26:31+00:00 Use the existing image.
2025-04-02 06:26:32+00:00 Generating deployment configuration.
2025-04-02 06:26:34+00:00 Submitting deployment to compute.
2025-04-02 06:26:41+00:00 Checking the status of deployment lr-prediction-service..
2025-04-02 06:28:20+00:00 Checking the status of inference endpoint lr-prediction-service.
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: fe4af5a7-c90e-4d14-aedb-e4693d1922f2
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: lr-prediction-service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image eba109211ca84988973a3a6b8f283707.azurecr.io/azureml/azureml_788d81cd6b656803aa87f41300150b20 locally. Please refer to https://aka.ms/debugimage#service-lau

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: fe4af5a7-c90e-4d14-aedb-e4693d1922f2
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: lr-prediction-service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image eba109211ca84988973a3a6b8f283707.azurecr.io/azureml/azureml_788d81cd6b656803aa87f41300150b20 locally. P

In [24]:
scoring_uri = service.scoring_uri

In [25]:
scoring_uri

'http://e9cddab6-15c5-4a4d-bdee-6101f526ee75.eastus.azurecontainer.io/score'